# EPJDS computations: missing links via hyperlink prediction (10 datasets)

This notebook runs the leakage-free hyperlink-prediction experiments and writes **all outputs** into the manuscript folders:

- Figures → `../figures/`
- Tables (LaTeX-ready) → `../tables/`
- Raw results / edgelists → `../data/`

**Dataset order used everywhere (tables/plots):**
1) Epstein (public email, Jmail-derived)  
2) Enron (public email, SNAP) — core-100  
3) EU-core (public email, SNAP)  
4) uni_email / URV (public email, Arenas)  
5–10) six covert-network proxies

All graphs are analyzed as **simple, undirected, unweighted** networks for comparability:
email graphs are symmetrized and time-aggregated.

**Methods computed**
Common Neighbors; Jaccard; Adamic–Adar; Resource Allocation; Preferential Attachment; Katz (truncated); Personalized PageRank (RWR); node2vec (optional); CHESHIRE (RF); Null-Tie; Matrix Completion.


In [1]:
# Imports
import os
import sys
import io
import gzip
import zipfile
import time
import math
import itertools
from pathlib import Path
from collections import OrderedDict, defaultdict

import numpy as np
import pandas as pd

import networkx as nx
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score, log_loss, matthews_corrcoef
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt

# Optional: node2vec baseline (gensim)
try:
    from gensim.models import Word2Vec
    _HAVE_GENSIM = True
except Exception:
    Word2Vec = None
    _HAVE_GENSIM = False

import urllib.request

In [2]:
# Configuration
RNG = np.random.default_rng(42)

# If you run from notebooks/, ROOT should be the project root; otherwise adjust.
CWD = Path.cwd()
ROOT = CWD.parent if (CWD.name.lower() in {"notebooks", "notebook"}) else CWD

FIG_DIR = ROOT / "figures"
TAB_DIR = ROOT / "tables"
DATA_DIR = ROOT / "data"
RAW_DIR  = DATA_DIR / "raw"
EDGE_DIR = DATA_DIR / "edgelists"

for d in [FIG_DIR, TAB_DIR, DATA_DIR, RAW_DIR, EDGE_DIR]:
    d.mkdir(parents=True, exist_ok=True)

# --- Email datasets (downloaded if missing) ---
# SNAP
ENRON_URL   = "https://snap.stanford.edu/data/email-Enron.txt.gz"
EUCORE_URL  = "https://snap.stanford.edu/data/email-Eu-core.txt.gz"
# Netzschleuder (CSV zip) for URV email
URV_URL     = "https://networks.skewed.de/net/uni_email/files/uni_email.csv.zip"

# Epstein CSV (you provide)
# We try several common locations; adjust if needed.
EPSTEIN_CSV_CANDIDATES = [
    ROOT / "Epstein_emails_from_jmail_ALL_chrs.csv",
    ROOT.parent / "Epstein_emails_from_jmail_ALL_chrs.csv",
    ROOT / "data" / "Epstein_emails_from_jmail_ALL_chrs.csv",
    ROOT.parent / "data" / "Epstein_emails_from_jmail_ALL_chrs.csv",
]
EPSTEIN_CSV = next((p for p in EPSTEIN_CSV_CANDIDATES if p.exists()), EPSTEIN_CSV_CANDIDATES[0])

# --- Experimental parameters ---
MISSING_RATES = [0.1, 0.3, 0.5]
NUM_TRIALS = 5

# To keep runtime bounded on larger graphs, we subsample training/testing hyperedges
MAX_TRAIN_HYPEREDGES = 5000
MAX_TEST_HYPEREDGES  = 5000

# Hyperedge definition: dyads + triangles (recommended for scalability)
USE_TRIANGLES = True

# node2vec baseline is optional (requires gensim). Also cap by size.
ENABLE_NODE2VEC = True
NODE2VEC_MAX_N = 5000

# Enron: induce a core subgraph for tractability
ENRON_CORE_K = 100

print("ROOT:", ROOT)
print("Epstein CSV:", EPSTEIN_CSV)
print("gensim available:", _HAVE_GENSIM)

ROOT: /Users/moses/WorkPlaces/Python Projects 2/0 0 EFrelated Missingness
Epstein CSV: /Users/moses/WorkPlaces/Python Projects 2/0 0 EFrelated Missingness/data/Epstein_emails_from_jmail_ALL_chrs.csv
gensim available: True


In [3]:

# ---------- Utilities ----------

def _download(url: str, dest: Path) -> Path:
    # Download url -> dest if dest does not exist.
    dest.parent.mkdir(parents=True, exist_ok=True)
    if dest.exists() and dest.stat().st_size > 0:
        return dest
    print(f"Downloading: {url} -> {dest}")
    urllib.request.urlretrieve(url, dest)
    return dest

def _read_txt_gz_edges(path: Path, sep=None, comment_prefix="#"):
    # Read SNAP-style .txt.gz edge list into a list of (u,v) strings.
    edges = []
    with gzip.open(path, "rt", encoding="utf-8", errors="ignore") as f:
        for line in f:
            line = line.strip()
            if not line or (comment_prefix and line.startswith(comment_prefix)):
                continue
            parts = line.split() if sep is None else line.split(sep)
            if len(parts) < 2:
                continue
            u, v = parts[0], parts[1]
            if u != v:
                edges.append((str(u), str(v)))
    return edges

def load_snap_txt_gz(url: str, name: str) -> Path:
    # Download SNAP txt.gz into data/raw/ and return local path.
    dest = RAW_DIR / f"{name}.txt.gz"
    return _download(url, dest)

def load_undirected_from_edges(edges):
    G = nx.Graph()
    G.add_edges_from(edges)
    G.remove_edges_from(nx.selfloop_edges(G))
    return G

def load_email_snap_graph(url: str, name: str, symmetrize: bool = True):
    path = load_snap_txt_gz(url, name)
    edges = _read_txt_gz_edges(path)
    # SNAP files list directed pairs; for our experiments we symmetrize to undirected
    if symmetrize:
        return load_undirected_from_edges(edges)
    else:
        G = nx.DiGraph()
        G.add_edges_from(edges)
        G.remove_edges_from(nx.selfloop_edges(G))
        return G

def load_csv_zip(url: str, name: str) -> pd.DataFrame:
    # Download a zip containing a CSV and return the dataframe.
    dest = RAW_DIR / f"{name}.csv.zip"
    _download(url, dest)
    with zipfile.ZipFile(dest, "r") as zf:
        members = [m for m in zf.namelist() if m.lower().endswith(".csv")]
        if not members:
            raise ValueError(f"No CSV found inside {dest}")
        with zf.open(members[0]) as f:
            data = f.read()
    df = pd.read_csv(io.BytesIO(data))
    return df

def load_uni_email_urv(url: str = URV_URL) -> nx.Graph:
    df = load_csv_zip(url, "uni_email_urv")
    if df.shape[1] < 2:
        raise ValueError("URV email CSV must have at least two columns.")
    ucol, vcol = df.columns[0], df.columns[1]
    edges = df[[ucol, vcol]].dropna()
    edges[ucol] = edges[ucol].astype(str)
    edges[vcol] = edges[vcol].astype(str)
    return load_undirected_from_edges(list(edges.itertuples(index=False, name=None)))

def load_edgelist_csv(path: Path, undirected=True, u="u", v="v"):
    df = pd.read_csv(path)
    if u not in df.columns or v not in df.columns:
        u, v = df.columns[:2]
    edges = df[[u, v]].dropna()
    edges[u] = edges[u].astype(str)
    edges[v] = edges[v].astype(str)
    if undirected:
        a = edges[[u, v]].min(axis=1)
        b = edges[[u, v]].max(axis=1)
        edges = pd.DataFrame({"u": a, "v": b})
        edges = edges[edges["u"] != edges["v"]]
        G = nx.Graph()
        G.add_edges_from(edges.itertuples(index=False, name=None))
    else:
        G = nx.DiGraph()
        G.add_edges_from(edges.itertuples(index=False, name=None))
        G.remove_edges_from(nx.selfloop_edges(G))
    return G

def graph_stats(G: nx.Graph):
    n = G.number_of_nodes()
    m = G.number_of_edges()
    dens = (2*m)/(n*(n-1)) if n > 1 else 0.0
    tri = int(sum(nx.triangles(G).values()) // 3) if isinstance(G, nx.Graph) else 0
    return {"n": n, "m": m, "density": dens, "triangles": tri}

def build_epstein_graph(csv_path: Path, year_min=2007, year_max=2025, undirected=True):
    df = pd.read_csv(csv_path)
    df = df.dropna(subset=["SENDER", "RECIPIENT", "DATE"])
    df["DATE"] = pd.to_datetime(df["DATE"], errors="coerce")
    df = df.dropna(subset=["DATE"])
    df = df[(df["DATE"].dt.year >= year_min) & (df["DATE"].dt.year <= year_max)]
    df["SENDER"] = df["SENDER"].astype(str)
    df["RECIPIENT"] = df["RECIPIENT"].astype(str)

    if undirected:
        a = df[["SENDER", "RECIPIENT"]].min(axis=1)
        b = df[["SENDER", "RECIPIENT"]].max(axis=1)
        pairs = pd.DataFrame({"u": a, "v": b})
        pairs = pairs[pairs["u"] != pairs["v"]]
        w = pairs.value_counts().reset_index(name="weight")
        G = nx.Graph()
        for _, r in w.iterrows():
            G.add_edge(r["u"], r["v"], weight=int(r["weight"]))
    else:
        w = df.groupby(["SENDER", "RECIPIENT"]).size().reset_index(name="weight")
        G = nx.DiGraph()
        for _, r in w.iterrows():
            if r["SENDER"] != r["RECIPIENT"]:
                G.add_edge(r["SENDER"], r["RECIPIENT"], weight=int(r["weight"]))
    return G, df

def induce_core_by_activity(G: nx.Graph, k=100):
    act = {}
    if any("weight" in d for _, _, d in G.edges(data=True)):
        for n in G.nodes():
            act[n] = sum(d.get("weight", 1.0) for _, _, d in G.edges(n, data=True))
    else:
        act = dict(G.degree())
    top = sorted(act.items(), key=lambda x: x[1], reverse=True)[:k]
    nodes = [n for n, _ in top]
    return G.subgraph(nodes).copy()

# ---- Hyperedges: dyads + triangles (recommended) ----
def triangle_hyperedges(G: nx.Graph):
    nodes = list(G.nodes())
    order = {n: i for i, n in enumerate(nodes)}
    neigh = {u: set(G.neighbors(u)) for u in nodes}
    tri = set()
    for u in nodes:
        Nu = neigh[u]
        for v in Nu:
            if order[v] <= order[u]:
                continue
            common = Nu.intersection(neigh.get(v, set()))
            for w in common:
                if order[w] <= order[v]:
                    continue
                tri.add(frozenset((u, v, w)))
    return list(tri)

def build_hyperedges(G: nx.Graph):
    dyads = [frozenset(e) for e in G.edges()]
    tris = triangle_hyperedges(G) if USE_TRIANGLES else []
    E = list(dict.fromkeys(dyads + tris))
    return E, dyads, tris

def sample_missing_hyperedges(E, missing_rate, rng):
    n = len(E)
    k = max(1, int(round(missing_rate * n)))
    idx = rng.choice(n, size=k, replace=False)
    miss = [E[i] for i in idx]
    obs = [E[i] for i in range(n) if i not in set(idx)]
    return obs, miss

def build_G_obs_from_missing(G: nx.Graph, E_miss):
    Gobs = G.copy()
    for S in E_miss:
        nodes = list(S)
        for i in range(len(nodes)):
            for j in range(i + 1, len(nodes)):
                if Gobs.has_edge(nodes[i], nodes[j]):
                    Gobs.remove_edge(nodes[i], nodes[j])
    return Gobs

def hyperdegree(E_obs, nodes):
    hd = {n: 0 for n in nodes}
    for S in E_obs:
        for n in S:
            hd[n] = hd.get(n, 0) + 1
    return hd

def induced_density(G, S):
    nodes = list(S)
    if len(nodes) <= 1:
        return 0.0
    H = G.subgraph(nodes)
    m = H.number_of_edges()
    n = H.number_of_nodes()
    return (2 * m) / (n * (n - 1)) if n > 1 else 0.0

def avg_clustering_on_nodes(G, S):
    nodes = list(S)
    if not nodes:
        return 0.0
    cl = nx.clustering(G, nodes=nodes)
    return float(np.mean(list(cl.values()))) if cl else 0.0

def avg_pairwise_metric(S, metric_func):
    nodes = list(S)
    if len(nodes) < 2:
        return 0.0
    s = 0.0
    cnt = 0
    for i in range(len(nodes)):
        for j in range(i + 1, len(nodes)):
            s += float(metric_func(nodes[i], nodes[j]))
            cnt += 1
    return s / cnt if cnt else 0.0

def hyperlink_features(G_obs, E_obs, S):
    nodes = list(S)
    deg = dict(G_obs.degree())
    hd = hyperdegree(E_obs, G_obs.nodes())
    cn = cn_score_factory(G_obs)
    feats = {
        "size": len(nodes),
        "avg_degree": float(np.mean([deg.get(n, 0) for n in nodes])) if nodes else 0.0,
        "induced_density": induced_density(G_obs, S),
        "avg_clustering": avg_clustering_on_nodes(G_obs, S),
        "avg_pair_cn": avg_pairwise_metric(S, cn),
        "avg_hyperdegree": float(np.mean([hd.get(n, 0) for n in nodes])) if nodes else 0.0,
    }
    return feats

def random_negative_hyperedges(all_nodes, sizes, existing_set, rng, max_tries=50000):
    all_nodes = list(all_nodes)
    sizes = list(sizes)
    neg = []
    tries = 0
    while len(neg) < len(sizes) and tries < max_tries:
        s = int(sizes[len(neg)])
        cand = frozenset(rng.choice(all_nodes, size=s, replace=False))
        tries += 1
        if cand in existing_set:
            continue
        neg.append(cand)
    return neg[: len(sizes)]

# ---- Pairwise baseline score factories ----
def cn_score_factory(G):
    neigh = {n: set(G.neighbors(n)) for n in G.nodes()}
    def cn(u, v):
        return len(neigh.get(u, set()).intersection(neigh.get(v, set())))
    return cn

def jaccard_score_factory(G):
    neigh = {n: set(G.neighbors(n)) for n in G.nodes()}
    def jac(u, v):
        Nu, Nv = neigh.get(u, set()), neigh.get(v, set())
        inter = len(Nu.intersection(Nv))
        uni = len(Nu.union(Nv))
        return inter / uni if uni else 0.0
    return jac

def aa_score_factory(G):
    neigh = {n: set(G.neighbors(n)) for n in G.nodes()}
    deg = dict(G.degree())
    def aa(u, v):
        inter = neigh.get(u, set()).intersection(neigh.get(v, set()))
        s = 0.0
        for w in inter:
            dw = deg.get(w, 0)
            if dw > 1:
                s += 1.0 / np.log(dw)
        return s
    return aa

def ra_score_factory(G):
    neigh = {n: set(G.neighbors(n)) for n in G.nodes()}
    deg = dict(G.degree())
    def ra(u, v):
        inter = neigh.get(u, set()).intersection(neigh.get(v, set()))
        s = 0.0
        for w in inter:
            dw = deg.get(w, 0)
            if dw > 0:
                s += 1.0 / dw
        return s
    return ra

def pa_score_factory(G):
    deg = dict(G.degree())
    def pa(u, v):
        return float(deg.get(u, 0) * deg.get(v, 0))
    return pa

def katz_pair_matrix(G: nx.Graph, beta=0.005, max_l=3):
    nodes = list(G.nodes())
    idx = {n: i for i, n in enumerate(nodes)}
    if not nodes:
        return nodes, idx, None
    A = nx.to_scipy_sparse_array(G, nodelist=nodes, dtype=float, format="csr")
    K = beta * A
    Apow = A
    for l in range(2, max_l + 1):
        Apow = Apow @ A
        K = K + (beta ** l) * Apow
    return nodes, idx, K

def ppr_score_factory(G: nx.Graph, alpha=0.85, max_iter=100, tol=1e-6):
    cache = {}
    nodes = list(G.nodes())
    base = {n: 0.0 for n in nodes}
    def ppr(u, v):
        if u not in cache:
            pers = dict(base)
            if u in pers:
                pers[u] = 1.0
            cache[u] = nx.pagerank(G, alpha=alpha, personalization=pers, max_iter=max_iter, tol=tol)
        return float(cache[u].get(v, 0.0))
    return ppr

def node2vec_embeddings(G: nx.Graph, dimensions=64, walk_length=20, num_walks=10, window=10, min_count=1, workers=1, seed=42):
    if not _HAVE_GENSIM:
        raise RuntimeError("gensim not available (install gensim to use node2vec baseline).")
    rng = np.random.default_rng(seed)
    nodes = list(G.nodes())
    neigh = {u: list(G.neighbors(u)) for u in nodes}
    walks = []
    for _ in range(num_walks):
        rng.shuffle(nodes)
        for start in nodes:
            walk = [str(start)]
            cur = start
            for _t in range(walk_length - 1):
                nbrs = neigh.get(cur, [])
                if not nbrs:
                    break
                cur = rng.choice(nbrs)
                walk.append(str(cur))
            walks.append(walk)
    model = Word2Vec(
        sentences=walks,
        vector_size=dimensions,
        window=window,
        min_count=min_count,
        sg=1,
        workers=workers,
        seed=seed,
        epochs=5
    )
    emb = {n: model.wv[str(n)] for n in G.nodes() if str(n) in model.wv}
    return emb

def embedding_score_factory(emb: dict):
    def score(u, v):
        if u not in emb or v not in emb:
            return 0.0
        return float(np.dot(emb[u], emb[v]))
    return score

def matrix_completion_pair_scores(G, rank=10):
    nodes = list(G.nodes())
    n = len(nodes)
    if n == 0:
        return lambda u, v: 0.0
    A = nx.to_numpy_array(G, nodelist=nodes, dtype=float)
    U, s, Vt = np.linalg.svd(A, full_matrices=False)
    r = min(rank, len(s))
    Ahat = (U[:, :r] * s[:r]) @ Vt[:r, :]
    idx = {n: i for i, n in enumerate(nodes)}
    def mc(u, v):
        iu, iv = idx.get(u, None), idx.get(v, None)
        if iu is None or iv is None:
            return 0.0
        return float(Ahat[iu, iv])
    return mc

def evaluate_trial(G, label, missing_rate, trial, rng):
    E_all, dyads, tris = build_hyperedges(G)
    E_obs, E_miss = sample_missing_hyperedges(E_all, missing_rate, rng)

    if len(E_miss) > MAX_TEST_HYPEREDGES:
        idx = rng.choice(len(E_miss), size=MAX_TEST_HYPEREDGES, replace=False)
        E_miss = [E_miss[i] for i in idx]

    G_obs = build_G_obs_from_missing(G, E_miss)
    existing_set = set(E_all)

    if len(E_obs) > MAX_TRAIN_HYPEREDGES:
        idx = rng.choice(len(E_obs), size=MAX_TRAIN_HYPEREDGES, replace=False)
        E_obs_train = [E_obs[i] for i in idx]
    else:
        E_obs_train = E_obs

    train_sizes = [len(S) for S in E_obs_train]
    neg_train = random_negative_hyperedges(G.nodes(), train_sizes, existing_set, rng)
    y = np.array([1] * len(E_obs_train) + [0] * len(neg_train))
    X = pd.DataFrame([hyperlink_features(G_obs, E_obs_train, S) for S in (E_obs_train + neg_train)])

    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25, random_state=trial, stratify=y)

    clf = RandomForestClassifier(n_estimators=300, random_state=trial, n_jobs=-1)
    clf.fit(X_train, y_train)

    test_sizes = [len(S) for S in E_miss]
    neg_test = random_negative_hyperedges(G.nodes(), test_sizes, existing_set, rng)
    X_test = pd.DataFrame([hyperlink_features(G_obs, E_obs_train, S) for S in (E_miss + neg_test)])
    y_test = np.array([1] * len(E_miss) + [0] * len(neg_test))

    p_cheshire = clf.predict_proba(X_test)[:, 1]

    cn = cn_score_factory(G_obs)
    jac = jaccard_score_factory(G_obs)
    aa = aa_score_factory(G_obs)
    ra = ra_score_factory(G_obs)
    pa = pa_score_factory(G_obs)

    _, idx_k, K = katz_pair_matrix(G_obs, beta=0.005, max_l=3)
    def katz(u, v):
        iu, iv = idx_k.get(u, None), idx_k.get(v, None)
        if iu is None or iv is None or K is None:
            return 0.0
        return float(K[iu, iv])

    ppr = ppr_score_factory(G_obs, alpha=0.85)
    mc = matrix_completion_pair_scores(G_obs, rank=10)

    have_n2v = bool(ENABLE_NODE2VEC and _HAVE_GENSIM and G_obs.number_of_nodes() <= NODE2VEC_MAX_N)
    if have_n2v:
        emb = node2vec_embeddings(G_obs, dimensions=64, walk_length=20, num_walks=10, window=10, workers=1, seed=42)
        n2v = embedding_score_factory(emb)

    def score_hyperedge(S, pair_score):
        return avg_pairwise_metric(S, pair_score)

    test_hyperedges = E_miss + neg_test
    scores = OrderedDict()
    scores["Null-Tie"] = np.full_like(y_test, 0.5, dtype=float)
    scores["Common Neighbors"] = np.array([score_hyperedge(S, cn) for S in test_hyperedges], dtype=float)
    scores["Jaccard"] = np.array([score_hyperedge(S, jac) for S in test_hyperedges], dtype=float)
    scores["Adamic–Adar"] = np.array([score_hyperedge(S, aa) for S in test_hyperedges], dtype=float)
    scores["Resource Allocation"] = np.array([score_hyperedge(S, ra) for S in test_hyperedges], dtype=float)
    scores["Preferential Attachment"] = np.array([score_hyperedge(S, pa) for S in test_hyperedges], dtype=float)
    scores["Katz"] = np.array([score_hyperedge(S, katz) for S in test_hyperedges], dtype=float)
    scores["Personalized PageRank"] = np.array([score_hyperedge(S, ppr) for S in test_hyperedges], dtype=float)
    if have_n2v:
        scores["node2vec"] = np.array([score_hyperedge(S, n2v) for S in test_hyperedges], dtype=float)
    scores["Matrix Completion"] = np.array([score_hyperedge(S, mc) for S in test_hyperedges], dtype=float)
    scores["CHESHIRE"] = p_cheshire

    for k, v in scores.items():
        if k in {"Null-Tie", "CHESHIRE"}:
            continue
        mn, mx = float(np.min(v)), float(np.max(v))
        denom = (mx - mn) if mx > mn else 1.0
        scores[k] = (v - mn) / denom

    rows = []
    for method, pred in scores.items():
        auc = roc_auc_score(y_test, pred) if len(np.unique(y_test)) > 1 else 0.5
        yhat = (pred >= 0.5).astype(int)
        rows.append({
            "dataset": label,
            "missing_rate": float(missing_rate),
            "trial": int(trial),
            "method": method,
            "ROC-AUC": float(auc),
            "F1": float(f1_score(y_test, yhat)),
            "Accuracy": float(accuracy_score(y_test, yhat)),
            "Log-loss": float(log_loss(y_test, np.clip(pred, 1e-9, 1 - 1e-9))),
            "MCC": float(matthews_corrcoef(y_test, yhat)),
            "n_nodes": int(G.number_of_nodes()),
            "n_edges": int(G.number_of_edges()),
            "n_hyperedges": int(len(E_all)),
            "n_test": int(len(E_miss))
        })
    return pd.DataFrame(rows)

def save_edgelist(G: nx.Graph, path: Path):
    df = pd.DataFrame(list(G.edges()), columns=["u", "v"])
    df.to_csv(path, index=False)

def _latex_clean(s: str) -> str:
    return str(s).replace("–", "--").replace("—", "--")

In [4]:
# ---------- Load covert-network proxy edgelists (6) ----------
proxy_files = [
    "christmas_eve_2000.csv",
    "bali_2002.csv",
    "australian_embassy_2004.csv",
    "bali_2005.csv",
    "hamburg_cell_911.csv",
    "london_gang.csv",
]

proxy_labels = {
    "christmas_eve_2000.csv": "Christmas Eve Bombings 2000 (proxy)",
    "bali_2002.csv": "Bali Bombing 2002 (proxy)",
    "australian_embassy_2004.csv": "Australian Embassy Bombing 2004 (proxy)",
    "bali_2005.csv": "Bali Bombing 2005 (proxy)",
    "hamburg_cell_911.csv": "Hamburg Cell 9/11 2001 (proxy)",
    "london_gang.csv": "London Gang 2005–2009 (proxy)",
}

proxies = OrderedDict()
for fn in proxy_files:
    path = EDGE_DIR / fn
    if not path.exists():
        raise FileNotFoundError(f"Missing edgelist: {path}")
    G = load_edgelist_csv(path, undirected=True)
    proxies[proxy_labels[fn]] = G

print("Loaded covert proxy datasets:", list(proxies.keys()))

Loaded covert proxy datasets: ['Christmas Eve Bombings 2000 (proxy)', 'Bali Bombing 2002 (proxy)', 'Australian Embassy Bombing 2004 (proxy)', 'Bali Bombing 2005 (proxy)', 'Hamburg Cell 9/11 2001 (proxy)', 'London Gang 2005–2009 (proxy)']


In [5]:
# ---------- Load email datasets + build final OrderedDict (10 datasets) ----------

# Epstein (Jmail-derived, local CSV)
if not EPSTEIN_CSV.exists():
    raise FileNotFoundError(f"Epstein CSV not found at: {EPSTEIN_CSV} (edit EPSTEIN_CSV_CANDIDATES in config)")
G_ep, _df_ep = build_epstein_graph(EPSTEIN_CSV, year_min=2007, year_max=2025, undirected=True)

# Enron (SNAP) — build full undirected, then core-100 by activity
G_en_full = load_email_snap_graph(ENRON_URL, "email-Enron", symmetrize=True)
G_en_core = induce_core_by_activity(G_en_full, k=ENRON_CORE_K)

# EU-core (SNAP) — undirected (symmetrized)
G_eu = load_email_snap_graph(EUCORE_URL, "email-Eu-core", symmetrize=True)

# URV uni_email (Netzschleuder)
G_urv = load_uni_email_urv(URV_URL)

# Save edgelists for reproducibility / downstream ERGM
save_edgelist(G_ep, EDGE_DIR / "epstein_emails_2007_2025_full.csv")
save_edgelist(G_en_core, EDGE_DIR / "enron_emails_core100.csv")
save_edgelist(G_eu, EDGE_DIR / "email_eu_core.csv")
save_edgelist(G_urv, EDGE_DIR / "uni_email_urv.csv")

# Final dataset order for ALL results/tables/plots
datasets = OrderedDict()
datasets["Epstein emails 2007–2025 (full)"] = G_ep
datasets["Enron emails (core-100)"] = G_en_core
datasets["EU-core emails (SNAP)"] = G_eu
datasets["uni_email (URV)"] = G_urv

# append covert proxies (keeps their internal order)
for k, v in proxies.items():
    datasets[k] = v

# Print summary stats
stats = []
for label, G in datasets.items():
    s = graph_stats(G)
    s["dataset"] = label
    stats.append(s)
pd.DataFrame(stats)[["dataset","n","m","density","triangles"]]

Downloading: https://snap.stanford.edu/data/email-Enron.txt.gz -> /Users/moses/WorkPlaces/Python Projects 2/0 0 EFrelated Missingness/data/raw/email-Enron.txt.gz
Downloading: https://snap.stanford.edu/data/email-Eu-core.txt.gz -> /Users/moses/WorkPlaces/Python Projects 2/0 0 EFrelated Missingness/data/raw/email-Eu-core.txt.gz
Downloading: https://networks.skewed.de/net/uni_email/files/uni_email.csv.zip -> /Users/moses/WorkPlaces/Python Projects 2/0 0 EFrelated Missingness/data/raw/uni_email_urv.csv.zip


,dataset,n,m,density,triangles
0,Epstein emails 2007–2025 (full),426,543,0.005998,161
1,Enron emails (core-100),100,1487,0.300404,9050
2,EU-core emails (SNAP),986,16064,0.033080,105461
3,uni_email (URV),1133,5451,0.008500,5343
4,Christmas Eve Bombings 2000 (proxy),14,16,0.175824,5
5,Bali Bombing 2002 (proxy),15,24,0.228571,22
6,Australian Embassy Bombing 2004 (proxy),10,15,0.333333,8
7,Bali Bombing 2005 (proxy),9,15,0.416667,11
8,Hamburg Cell 9/11 2001 (proxy),12,23,0.348485,23
9,London Gang 2005–2009 (proxy),50,85,0.069388,46


In [6]:
# ---------- Run missingness experiments ----------
all_rows = []
t0 = time.time()

for label, G in datasets.items():
    for mr in MISSING_RATES:
        for t in range(NUM_TRIALS):
            df_trial = evaluate_trial(G, label, mr, t, RNG)
            all_rows.append(df_trial)
            print(f"Done: {label} | mr={mr} | trial={t} | rows={len(df_trial)}")

results = pd.concat(all_rows, ignore_index=True)
results.to_csv(DATA_DIR / "all_results_raw_10datasets.csv", index=False)

print("Total rows:", results.shape[0], "Elapsed (s):", round(time.time() - t0, 2))
results.head()

Done: Epstein emails 2007–2025 (full) | mr=0.1 | trial=0 | rows=11
Done: Epstein emails 2007–2025 (full) | mr=0.1 | trial=1 | rows=11
Done: Epstein emails 2007–2025 (full) | mr=0.1 | trial=2 | rows=11
Done: Epstein emails 2007–2025 (full) | mr=0.1 | trial=3 | rows=11
Done: Epstein emails 2007–2025 (full) | mr=0.1 | trial=4 | rows=11
Done: Epstein emails 2007–2025 (full) | mr=0.3 | trial=0 | rows=11
Done: Epstein emails 2007–2025 (full) | mr=0.3 | trial=1 | rows=11
Done: Epstein emails 2007–2025 (full) | mr=0.3 | trial=2 | rows=11
Done: Epstein emails 2007–2025 (full) | mr=0.3 | trial=3 | rows=11
Done: Epstein emails 2007–2025 (full) | mr=0.3 | trial=4 | rows=11
Done: Epstein emails 2007–2025 (full) | mr=0.5 | trial=0 | rows=11
Done: Epstein emails 2007–2025 (full) | mr=0.5 | trial=1 | rows=11
Done: Epstein emails 2007–2025 (full) | mr=0.5 | trial=2 | rows=11
Done: Epstein emails 2007–2025 (full) | mr=0.5 | trial=3 | rows=11
Done: Epstein emails 2007–2025 (full) | mr=0.5 | trial=4 | row

Done: Hamburg Cell 9/11 2001 (proxy) | mr=0.5 | trial=1 | rows=11
Done: Hamburg Cell 9/11 2001 (proxy) | mr=0.5 | trial=2 | rows=11
Done: Hamburg Cell 9/11 2001 (proxy) | mr=0.5 | trial=3 | rows=11
Done: Hamburg Cell 9/11 2001 (proxy) | mr=0.5 | trial=4 | rows=11
Done: London Gang 2005–2009 (proxy) | mr=0.1 | trial=0 | rows=11
Done: London Gang 2005–2009 (proxy) | mr=0.1 | trial=1 | rows=11
Done: London Gang 2005–2009 (proxy) | mr=0.1 | trial=2 | rows=11
Done: London Gang 2005–2009 (proxy) | mr=0.1 | trial=3 | rows=11
Done: London Gang 2005–2009 (proxy) | mr=0.1 | trial=4 | rows=11
Done: London Gang 2005–2009 (proxy) | mr=0.3 | trial=0 | rows=11
Done: London Gang 2005–2009 (proxy) | mr=0.3 | trial=1 | rows=11
Done: London Gang 2005–2009 (proxy) | mr=0.3 | trial=2 | rows=11
Done: London Gang 2005–2009 (proxy) | mr=0.3 | trial=3 | rows=11
Done: London Gang 2005–2009 (proxy) | mr=0.3 | trial=4 | rows=11
Done: London Gang 2005–2009 (proxy) | mr=0.5 | trial=0 | rows=11
Done: London Gang 200

,dataset,missing_rate,trial,method,ROC-AUC,F1,Accuracy,Log-loss,MCC,n_nodes,n_edges,n_hyperedges,n_test
0,Epstein emails 2007–2025 (full),0.1,0,Null-Tie,0.500000,0.666667,0.500000,0.693147,0.000000,426,543,704,70
1,Epstein emails 2007–2025 (full),0.1,0,Common Neighbors,0.526735,0.292683,0.585714,6.584479,0.306186,426,543,704,70
2,Epstein emails 2007–2025 (full),0.1,0,Jaccard,0.431020,0.045977,0.407143,8.316134,-0.284293,426,543,704,70
3,Epstein emails 2007–2025 (full),0.1,0,Adamic–Adar,0.555714,0.292683,0.585714,6.581107,0.306186,426,543,704,70
4,Epstein emails 2007–2025 (full),0.1,0,Resource Allocation,0.553878,0.250000,0.571429,6.681741,0.277350,426,543,704,70


In [7]:
# ---------- Summaries + LaTeX tables (written to ../tables/) ----------
METHOD_ORDER = [
    "CHESHIRE",
    "Matrix Completion",
    "node2vec",
    "Personalized PageRank",
    "Katz",
    "Preferential Attachment",
    "Resource Allocation",
    "Adamic–Adar",
    "Jaccard",
    "Common Neighbors",
    "Null-Tie",
]

# enforce dataset order
dataset_order = list(datasets.keys())
results["dataset"] = pd.Categorical(results["dataset"], categories=dataset_order, ordered=True)
results["method"] = pd.Categorical(results["method"], categories=METHOD_ORDER, ordered=True)

def write_table(tex_path: Path, body: str):
    tex_path.parent.mkdir(parents=True, exist_ok=True)
    tex_path.write_text(body, encoding="utf-8")
    print("Wrote:", tex_path)

def latex_table(df: pd.DataFrame, caption: str, label: str, *, longtable=False, fontsize="\small"):
    df2 = df.copy()
    # clean dashes for LaTeX
    if "dataset" in df2.columns:
        df2["dataset"] = df2["dataset"].astype(str).map(_latex_clean)
    if "method" in df2.columns:
        df2["method"] = df2["method"].astype(str).map(_latex_clean)

    latex = df2.to_latex(index=False, escape=False, longtable=longtable)
    if longtable:
        # longtable already has its own environment
        out = f"""{fontsize}
{latex}
"""
    else:
        out = f"""\begin{{table}}[t]
{fontsize}
\centering
\caption{{{caption}}}
\label{{{label}}}
{latex}
\end{{table}}
"""
    return out

# Network stats
net_rows = []
for label, G in datasets.items():
    s = graph_stats(G)
    net_rows.append({
        "dataset": label,
        "nodes": s["n"],
        "edges": s["m"],
        "density": round(s["density"], 4),
        "triangles": s["triangles"],
    })
net_df = pd.DataFrame(net_rows)
write_table(TAB_DIR / "network_stats.tex",
            latex_table(net_df, "Summary statistics for the 10 interaction graphs (undirected, unweighted).", "tab:network_stats"))

# Overall performance summary (mean over datasets, missing rates, trials)
overall = (results
           .groupby("method", observed=True)["ROC-AUC"]
           .agg(["mean","std","count"])
           .reset_index())
overall["SE"] = overall["std"] / np.sqrt(overall["count"])
overall = overall.sort_values("mean", ascending=False)
overall_disp = overall[["method","mean","SE"]].copy()
overall_disp["mean"] = overall_disp["mean"].map(lambda x: f"{x:.3f}")
overall_disp["SE"] = overall_disp["SE"].map(lambda x: f"{x:.3f}")
write_table(TAB_DIR / "overall_summary_table.tex",
            latex_table(overall_disp, "Overall ROC--AUC (mean $\pm$ SE) across datasets, missing rates, and trials.", "tab:overall_summary"))

# Mean ROC-AUC by dataset and method at rho=0.3
rho = 0.3
hm = (results[results["missing_rate"] == rho]
      .groupby(["dataset","method"], observed=True)["ROC-AUC"]
      .mean()
      .reset_index())
hm_piv = hm.pivot(index="dataset", columns="method", values="ROC-AUC").reset_index()
# keep method order columns
cols = ["dataset"] + [m for m in METHOD_ORDER if m in hm_piv.columns]
hm_piv = hm_piv[cols]
# format
for c in cols[1:]:
    hm_piv[c] = hm_piv[c].map(lambda x: "" if pd.isna(x) else f"{x:.3f}")
write_table(TAB_DIR / "rocauc_by_network_table.tex",
            latex_table(hm_piv, f"Mean ROC--AUC by dataset and method at missing rate $\\rho={rho}$.", "tab:rocauc_by_network"))

# ROC-AUC vs missing rate (mean over datasets + trials)
mr = (results
      .groupby(["method","missing_rate"], observed=True)["ROC-AUC"]
      .mean()
      .reset_index())
mr_piv = mr.pivot(index="method", columns="missing_rate", values="ROC-AUC").reset_index()
# reorder methods
mr_piv["method"] = pd.Categorical(mr_piv["method"], categories=METHOD_ORDER, ordered=True)
mr_piv = mr_piv.sort_values("method")
# format columns
mr_piv_cols = ["method"] + sorted([c for c in mr_piv.columns if c != "method"])
mr_piv = mr_piv[mr_piv_cols]
for c in mr_piv_cols[1:]:
    mr_piv[c] = mr_piv[c].map(lambda x: "" if pd.isna(x) else f"{x:.3f}")
mr_piv = mr_piv.rename(columns={c: f"$\\rho={c}$" for c in mr_piv_cols[1:]})
write_table(TAB_DIR / "auc_by_missing_rate_table.tex",
            latex_table(mr_piv, "Mean ROC--AUC as a function of missing rate $\\rho$ (averaged across datasets and trials).", "tab:auc_by_missing_rate"))

# Detailed longtable (appendix): per dataset, missing rate, method
det = (results
       .groupby(["dataset","missing_rate","method"], observed=True)["ROC-AUC"]
       .agg(["mean","std","count"])
       .reset_index())
det["SE"] = det["std"] / np.sqrt(det["count"])
det_disp = det[["dataset","missing_rate","method","mean","SE"]].copy()
det_disp["missing_rate"] = det_disp["missing_rate"].map(lambda x: f"{x:.1f}")
det_disp["mean"] = det_disp["mean"].map(lambda x: f"{x:.3f}")
det_disp["SE"] = det_disp["SE"].map(lambda x: f"{x:.3f}")
write_table(TAB_DIR / "detailed_results_longtable.tex",
            latex_table(det_disp, "Detailed ROC--AUC (mean $\pm$ SE) by dataset, missing rate, and method.", "tab:detailed_results", longtable=True))

Wrote: /Users/moses/WorkPlaces/Python Projects 2/0 0 EFrelated Missingness/tables/network_stats.tex
Wrote: /Users/moses/WorkPlaces/Python Projects 2/0 0 EFrelated Missingness/tables/overall_summary_table.tex
Wrote: /Users/moses/WorkPlaces/Python Projects 2/0 0 EFrelated Missingness/tables/rocauc_by_network_table.tex
Wrote: /Users/moses/WorkPlaces/Python Projects 2/0 0 EFrelated Missingness/tables/auc_by_missing_rate_table.tex
Wrote: /Users/moses/WorkPlaces/Python Projects 2/0 0 EFrelated Missingness/tables/detailed_results_longtable.tex


In [8]:
# ---------- Plots (written to ../figures/) ----------

# Mean ROC-AUC by missing rate (averaged across datasets & trials)
agg_mr = (results
          .groupby(["method","missing_rate"], observed=True)["ROC-AUC"]
          .mean()
          .reset_index())
agg_mr["method"] = pd.Categorical(agg_mr["method"], categories=METHOD_ORDER, ordered=True)
agg_mr = agg_mr.sort_values(["method","missing_rate"])

plt.figure()
for method in METHOD_ORDER:
    d = agg_mr[agg_mr["method"] == method]
    if d.empty:
        continue
    plt.plot(d["missing_rate"], d["ROC-AUC"], marker="o", label=method)
plt.xlabel("Missing rate $\\rho$")
plt.ylabel("Mean ROC-AUC")
plt.title("Performance vs missing rate (mean over datasets & trials)")
plt.legend(bbox_to_anchor=(1.02, 1), loc="upper left", fontsize=8)
plt.tight_layout()
plt.savefig(FIG_DIR / "performance_vs_missing_rate.png", dpi=200)
plt.close()

# Heatmap: mean ROC-AUC by dataset and method at rho=0.3
rho = 0.3
hm = (results[results["missing_rate"] == rho]
      .groupby(["dataset","method"], observed=True)["ROC-AUC"]
      .mean()
      .reset_index())
hm_piv = hm.pivot(index="dataset", columns="method", values="ROC-AUC")
# ensure ordered axes
hm_piv = hm_piv.reindex(index=dataset_order, columns=[m for m in METHOD_ORDER if m in hm_piv.columns])

plt.figure(figsize=(12, 6))
im = plt.imshow(hm_piv.values, aspect="auto")
plt.colorbar(im, fraction=0.046, pad=0.04)
plt.xticks(range(hm_piv.shape[1]), [m.replace("–","-") for m in hm_piv.columns], rotation=45, ha="right", fontsize=8)
plt.yticks(range(hm_piv.shape[0]), [d.replace("–","-") for d in hm_piv.index], fontsize=8)
plt.title(f"Mean ROC-AUC by dataset and method at $\\rho={rho}$")
plt.tight_layout()
plt.savefig(FIG_DIR / "performance_by_network_heatmap.png", dpi=200)
plt.close()

print("Saved figures to:", FIG_DIR)

Saved figures to: /Users/moses/WorkPlaces/Python Projects 2/0 0 EFrelated Missingness/figures


In [10]:
import os, zipfile, time

stamp = time.strftime("%Y%m%d-%H%M")
zip_path = f"run_outputs_{stamp}.zip"

paths = ["figures", "tables", "data"]
with zipfile.ZipFile(zip_path, "w", compression=zipfile.ZIP_DEFLATED) as z:
    for p in paths:
        for root, _, files in os.walk(p):
            for fn in files:
                fp = os.path.join(root, fn)
                z.write(fp, fp)

print("Created:", zip_path)

Created: run_outputs_20260109-0221.zip


# ---------- Controlled scaling simulation (synthetic graphs) ----------
# Produces:
# - ../figures/simulation_auc_vs_n.png
# - ../figures/simulation_cheshire_fit_time.png
# - ../tables/scaling_table.tex

def make_sbm(n, k=4, p_in=0.05, p_out=0.005, seed=0):
    rng = np.random.default_rng(seed)
    sizes = [n // k] * k
    sizes[0] += n - sum(sizes)
    probs = [[p_out]*k for _ in range(k)]
    for i in range(k):
        probs[i][i] = p_in
    G = nx.stochastic_block_model(sizes, probs, seed=seed)
    # simple undirected graph
    G = nx.Graph(G)
    G.remove_edges_from(nx.selfloop_edges(G))
    # relabel nodes to strings for consistency
    return nx.relabel_nodes(G, {u: str(u) for u in G.nodes()})

def cheshire_fit_time_and_auc(G, missing_rate=0.3, trial=0, rng=None):
    rng = RNG if rng is None else rng
    E_all, _, _ = build_hyperedges(G)
    E_obs, E_miss = sample_missing_hyperedges(E_all, missing_rate, rng)

    if len(E_miss) > MAX_TEST_HYPEREDGES:
        idx = rng.choice(len(E_miss), size=MAX_TEST_HYPEREDGES, replace=False)
        E_miss = [E_miss[i] for i in idx]

    G_obs = build_G_obs_from_missing(G, E_miss)
    existing_set = set(E_all)

    if len(E_obs) > MAX_TRAIN_HYPEREDGES:
        idx = rng.choice(len(E_obs), size=MAX_TRAIN_HYPEREDGES, replace=False)
        E_obs_train = [E_obs[i] for i in idx]
    else:
        E_obs_train = E_obs

    train_sizes = [len(S) for S in E_obs_train]
    neg_train = random_negative_hyperedges(G.nodes(), train_sizes, existing_set, rng)
    y = np.array([1] * len(E_obs_train) + [0] * len(neg_train))
    X = pd.DataFrame([hyperlink_features(G_obs, E_obs_train, S) for S in (E_obs_train + neg_train)])
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25, random_state=trial, stratify=y)

    clf = RandomForestClassifier(n_estimators=300, random_state=trial, n_jobs=-1)
    t0 = time.time()
    clf.fit(X_train, y_train)
    fit_time = time.time() - t0

    test_sizes = [len(S) for S in E_miss]
    neg_test = random_negative_hyperedges(G.nodes(), test_sizes, existing_set, rng)
    X_test = pd.DataFrame([hyperlink_features(G_obs, E_obs_train, S) for S in (E_miss + neg_test)])
    y_test = np.array([1] * len(E_miss) + [0] * len(neg_test))

    p = clf.predict_proba(X_test)[:, 1]
    auc = roc_auc_score(y_test, p) if len(np.unique(y_test)) > 1 else 0.5
    return float(auc), float(fit_time)

sizes = [200, 500, 1000, 2000]
sim_rows = []
for n in sizes:
    for t in range(3):
        Gs = make_sbm(n, seed=100 + t)
        auc, ft = cheshire_fit_time_and_auc(Gs, missing_rate=0.3, trial=t)
        sim_rows.append({"n": n, "trial": t, "ROC-AUC": auc, "fit_time_s": ft})
        print("SBM n=", n, "trial=", t, "AUC=", round(auc,3), "fit_time=", round(ft,2), "s")

sim = pd.DataFrame(sim_rows)
sim_summary = sim.groupby("n").agg({"ROC-AUC":["mean","std"], "fit_time_s":["mean","std"]})
sim_summary.columns = ["_".join(c) for c in sim_summary.columns]
sim_summary = sim_summary.reset_index()

# plots
plt.figure()
plt.plot(sim_summary["n"], sim_summary["ROC-AUC_mean"], marker="o")
plt.xlabel("n (nodes)")
plt.ylabel("CHESHIRE ROC-AUC (mean)")
plt.title("Scaling: CHESHIRE AUC vs n (SBM, ρ=0.3)")
plt.tight_layout()
plt.savefig(FIG_DIR / "simulation_auc_vs_n.png", dpi=200)
plt.close()

plt.figure()
plt.plot(sim_summary["n"], sim_summary["fit_time_s_mean"], marker="o")
plt.xlabel("n (nodes)")
plt.ylabel("CHESHIRE fit time (s, mean)")
plt.title("Scaling: CHESHIRE fit time vs n (SBM, ρ=0.3)")
plt.tight_layout()
plt.savefig(FIG_DIR / "simulation_cheshire_fit_time.png", dpi=200)
plt.close()

# scaling table
tab = sim_summary.copy()
tab["ROC-AUC_mean"] = tab["ROC-AUC_mean"].map(lambda x: f"{x:.3f}")
tab["ROC-AUC_std"] = tab["ROC-AUC_std"].map(lambda x: f"{x:.3f}")
tab["fit_time_s_mean"] = tab["fit_time_s_mean"].map(lambda x: f"{x:.2f}")
tab["fit_time_s_std"] = tab["fit_time_s_std"].map(lambda x: f"{x:.2f}")
tab = tab.rename(columns={
    "n": "nodes",
    "ROC-AUC_mean": "ROC-AUC (mean)",
    "ROC-AUC_std": "ROC-AUC (sd)",
    "fit_time_s_mean": "fit time s (mean)",
    "fit_time_s_std": "fit time s (sd)",
})
write_table(TAB_DIR / "scaling_table.tex",
            latex_table(tab, "Synthetic scaling results (SBM, $\\rho=0.3$): CHESHIRE ROC--AUC and training time.", "tab:scaling"))

print("Saved scaling figures + table.")
